# Multiview Training
This code will produce a model that can take two views of the cow and hopfully achieve a better predication. 

In [65]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
import keras
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from tqdm.notebook import tqdm
import keract
from matplotlib import pyplot as plt
import cv2

In [36]:
DATABASE_LOCATION = 'C:/Users/derek/Documents/cattle_data'
INPUT_PATH = DATABASE_LOCATION+'/training'
DATASET = 'two_view_training.json'

## Load Data

In [42]:
# load training data
import pandas as pd
df = pd.read_json(os.path.join(INPUT_PATH, DATASET))
print(len(df))
df.head()

19725


,date,weight,side_id,end_id,side_path,end_path
0,2021-11-30,565,126454,126452,C:/Users/derek/Documents/cattle_data/training/...,C:/Users/derek/Documents/cattle_data/training/...
1,2021-11-30,565,126463,126451,C:/Users/derek/Documents/cattle_data/training/...,C:/Users/derek/Documents/cattle_data/training/...
2,2021-11-30,565,126453,126461,C:/Users/derek/Documents/cattle_data/training/...,C:/Users/derek/Documents/cattle_data/training/...
3,2021-11-30,565,126460,126462,C:/Users/derek/Documents/cattle_data/training/...,C:/Users/derek/Documents/cattle_data/training/...
4,2021-11-30,565,126449,126464,C:/Users/derek/Documents/cattle_data/training/...,C:/Users/derek/Documents/cattle_data/training/...


In [43]:
# Add padding to image 
# Function taken from:
# https://note.nkmk.me/en/python-pillow-add-margin-expand-canvas/
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [45]:
# get paths to all images
paths = []

# add paths
for p in os.listdir(os.path.join(INPUT_PATH, 'two_views')):
    paths.append(os.path.join(INPUT_PATH, 'two_views', p))
    
print(len(paths))
print(paths[0])

38397
C:/Users/derek/Documents/cattle_data/training\two_views\end_100477.png


In [46]:
# preprocess entire dataset
pbar = tqdm(total=len(paths))

for path in paths:
    
    img = Image.open(path)
    img = expand2square(img, (0)).resize((224,224))
    img.save(path)
    pbar.update(1)

  0%|          | 0/38397 [00:00<?, ?it/s]

In [47]:
# trim the data to a constrained weight range 
df_trim = df[(df.weight > 400) & (df.weight < 1450)]

In [48]:
# reserve recent data for evaluation
# last reserve from 2022-01-22
cuttoff = '2022-02-13'
eval_df = df_trim[df_trim.date > cuttoff].copy()
training_df = df_trim[df_trim.date < cuttoff].copy()

In [51]:
len(eval_df)

2941

In [49]:
# split the training data into training and testing
def train_test_split(d, frac=0.2):
    test = d.sample(frac=frac, axis=0)
    train = d.drop(index=test.index).sample(frac=1, axis=0)
    return train, test

In [50]:
train_df, test_df = train_test_split(training_df)

In [63]:
def create_gen(model_input_shape, color='grayscale'):
    
    train_datagen = image.ImageDataGenerator(
            rescale=1./255,
            #shear_range=0.2,
            zoom_range=0.1,
            horizontal_flip=True
            )

    test_datagen = image.ImageDataGenerator(rescale=1./255)

    
    train_side_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=os.path.join(INPUT_PATH, 'img'),
            x_col=('side_path', 'end_path'),
            y_col='weight',
            target_size=model_input_shape,
            batch_size=128,
            class_mode='raw',
            color_mode=color,
            )
 
    validation_side_generator = test_datagen.flow_from_dataframe(
            dataframe=test_df,
            directory=os.path.join(INPUT_PATH, 'img'),
            x_col=('side_path', 'end_path'),
            y_col='weight',
            target_size=model_input_shape,
            batch_size=128,
            class_mode='raw',
            color_mode=color,
            )

    return train_side_generator, train_end_generator, validation_side_generator, validation_end_generator

In [ ]:
def augment(images, horizantal_flip=True, zoom_range=0.1):
    if horizantal_flip:
        images = np.flip(images, axis=1)
        
    

In [ ]:
def flow_from_frame()

In [ ]:
class Generator():
    """
    A Custom image data generator to supply two image inputs to a model.
    The images flow from a dataframe path. 
    """
    def __init__(self, model_input_shape, color='grayscale'):
        
        
        
    def flow_from_frame(d, x_col, y_col, )
    

In [64]:
train_gen, val_gen = create_gen((224,224), color='rgb')

KeyError: ('side_path', 'end_path')

In [60]:
image.ImageDataGenerator.flow_from_dataframe?

Signature:
image.ImageDataGenerator.flow_from_dataframe(
    self,
    dataframe,
    directory=None,
    x_col='filename',
    y_col='class',
    weight_col=None,
    target_size=(256, 256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    subset=None,
    interpolation='nearest',
    validate_filenames=True,
    **kwargs,
)
Docstring:
Takes the dataframe and the path to a directory + generates batches.

 The generated batches contain augmented/normalized data.

**A simple tutorial can be found **[here](
                            http://bit.ly/keras_flow_from_dataframe).

Args:
    dataframe: Pandas dataframe containing the filepaths relative to
      `directory` (or absolute paths if `directory` is None) of the images
      in a string column. It should include other column/s
        depending on the `class_mode`:
        - if `class_mode` is

## Create Two View Model

In [18]:
# load the model pretrained on single images
side_model = keras.models.load_model(DATABASE_LOCATION+"/models/exceptionet_best_model")
end_model = keras.models.load_model(DATABASE_LOCATION+"/models/exceptionet_best_model")

In [19]:
side_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 100)               204900    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                           

In [20]:
# unfreeze layers
for layer in side_model.layers:
    layer.trainable = True
for layer in end_model.layers:
    layer.trainable = True

In [24]:
# take off last layer
side_out = side_model.layers[-2].output
end_out = end_model.layers[-2].output

side_model = tf.keras.Model(inputs = side_model.input, outputs = side_out)
end_model = tf.keras.Model(inputs = end_model.input, outputs = end_out)

In [32]:
# create new inputs for side and end
side_input_layer = tf.keras.Input(shape=(224, 224, 3))
end_input_layer = tf.keras.Input(shape=(224, 224, 3))

side = side_model(side_input_layer, training=True)
end = end_model(end_input_layer, training=True)

# combine side and end features 
x = tf.keras.layers.concatenate([side, end])
#x = tf.keras.layers.Activation('relu')(x)

bias_init = tf.keras.initializers.Constant(700.0)#df.weight.mean())
outputs = tf.keras.layers.Dense(1, bias_initializer=bias_init)(x) # reduce to one node for weight prediction

model_xcept = tf.keras.Model(inputs = [side_input_layer, end_input_layer], outputs = outputs)

In [33]:
model_xcept.compile(
    loss= "mean_squared_error" , 
    optimizer="adam", 
    metrics=["mean_squared_error"])

In [34]:
model_xcept.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 100)          21066380    ['input_10[0][0]']               
                                                                                            

## Train the Model

In [57]:
hist = model_xcept.fit(
        [train_side_gen, train_end_gen],
        steps_per_epoch=300,
        epochs=100,
        validation_data=[val_side_gen, val_end_gen],
        validation_steps=25,
        #callbacks=[tensorboard_callback]
        )

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'keras.preprocessing.image.DataFrameIterator'>"}), <class 'NoneType'>

In [59]:
model_xcept.fit?

Signature:
model_xcept.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Docstring:
Trains the model for a fixed number of epochs (iterations on a dataset).

Args:
    x: Input data. It could be:
      - A Numpy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
      - A TensorFlow tensor, or a list of tensors
        (in case the model has multiple inputs).
      - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
      - A `tf.data` dataset. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, target

In [58]:
type(train_end_gen)

keras.preprocessing.image.DataFrameIterator

In [ ]:
model_vit.save(DATABASE_LOCATION+"/models/vit_best_model")

In [ ]:
history = hist
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()